In [4]:
"""Demonstrates molecular dynamics for isolated particles."""
from ase.cluster.cubic import FaceCenteredCubic
from ase.optimize import QuasiNewton
from ase.md.velocitydistribution import (MaxwellBoltzmannDistribution,
                                         Stationary, ZeroRotation)
from ase.md.verlet import VelocityVerlet
from ase import units
from ase.calculators.emt import EMT
from ase.io import read
from ase.calculators.lammpsrun import LAMMPS

atoms = read('moldyn4.traj')

parameters = {'pair_style': 'aenet Cu.nn',
              'pair_coeff': ['* *']}

lammps = LAMMPS(parameters=parameters)

# Describe the interatomic interactions with the Effective Medium Theory
atoms.calc = lammps

print(atoms.get_potential_energy())

# Do a quick relaxation of the cluster

# Set the momenta corresponding to T=1200K
MaxwellBoltzmannDistribution(atoms, temperature_K=1200)
Stationary(atoms)  # zero linear momentum
ZeroRotation(atoms)  # zero angular momentum

# We want to run MD using the VelocityVerlet algorithm.

# Save trajectory:
dyn = VelocityVerlet(atoms, 5 * units.fs, trajectory='moldyn5.traj')


def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))


dyn.attach(printenergy, interval=10)

# Now run the dynamics
printenergy()
dyn.run(2000)

/nethome/farri002/miniconda3/envs/aims/lib/python3.11/site-packages/ase/calculators/lammpsrun.py:191: UserWarning: You are using an old syntax to set 'parameters'.
Please use LAMMPSRUN.set().
  warnings.warn(self.legacy_warn_string.format("parameters"))
Exception in thread Thread-6 (read_lammps_log):
Traceback (most recent call last):
  File "/nethome/farri002/miniconda3/envs/aims/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/nethome/farri002/miniconda3/envs/aims/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/nethome/farri002/miniconda3/envs/aims/lib/python3.11/site-packages/ase/calculators/lammpsrun.py", line 586, in read_lammps_log
    raise RuntimeError(f'LAMMPS exits with error message: {line}')
RuntimeError: LAMMPS exits with error message: ERROR: Did not assign all atoms correctly (../read_data.cpp:1258)



RuntimeError: Failed to retrieve any thermo_style-output